In [160]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/rome
git clone https://github.com/kmeng01/rome rome > install.log 2>&1
pip install -r /content/rome/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [161]:
IS_COLAB = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/rome")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

In [162]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
import os, re, json
import torch, numpy
from collections import defaultdict
from util import nethook
from util.globals import DATA_DIR
from experiments.causal_trace import (
    ModelAndTokenizer,
    layername,
    guess_subject,
    plot_trace_heatmap,
)
from experiments.causal_trace import (
    make_inputs,
    decode_tokens,
    find_token_range,
    predict_token,
    predict_from_input,
    collect_embedding_std,
)
from dsets import KnownsDataset

torch.set_grad_enabled(False)

In [164]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Jan  8 13:40:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:01:00.0 Off |                  Off |
| 61

In [165]:
model_name = "EleutherAI/gpt-j-6B"  # or "EleutherAI/gpt-j-6B" or "EleutherAI/gpt-neox-20b"
mt = ModelAndTokenizer(
    model_name,
    low_cpu_mem_usage=True,
    torch_dtype=(torch.float16 if "20b" in model_name else None),
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 47.54 GiB total capacity; 39.76 GiB already allocated; 66.75 MiB free; 42.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
predict_token(
    mt,
    ["Megan Rapinoe plays the sport of", "The Space Needle is in the city of"],
    return_p=True,
)

In [ ]:
noise_level = 0.1

In [ ]:
def trace_with_patch(
    model,  # The model
    inp,  # A set of inputs
    states_to_patch,  # A list of (token index, layername) triples to restore
    answers_t,  # Answer probabilities to collect
    tokens_to_mix,  # Range of tokens to corrupt (begin, end)
    result_dict,
    noise=0.1,  # Level of noise to add
    trace_layers=None,  # List of traced outputs to return
):
    prng = numpy.random.RandomState(1)  # For reproducibility, use pseudorandom noise
    patch_spec = defaultdict(list)
    for t, l in states_to_patch:
        patch_spec[l].append(t)
    embed_layername = layername(model, 0, "embed")

    def untuple(x):
        return x[0] if isinstance(x, tuple) else x

    # Define the model-patching rule.
    def patch_rep(x, layer):
        if layer == embed_layername:
            # If requested, we corrupt a range of token embeddings on batch items x[1:]
            if tokens_to_mix is not None:
                b, e = tokens_to_mix
                x[1:, b:e] += noise * torch.from_numpy(
                    prng.randn(x.shape[0] - 1, e - b, x.shape[2])
                ).to(x.device)
            return x
        if layer not in patch_spec:
            return x
        # If this layer is in the patch_spec, restore the uncorrupted hidden state
        # for selected tokens.
        h = untuple(x)
        for t in patch_spec[layer]:
            h[1:, t] = h[0, t]
        return x

    # With the patching rules defined, run the patched model in inference.
    additional_layers = [] if trace_layers is None else trace_layers
    with torch.no_grad(), nethook.TraceDict(
        model,
        [embed_layername] + list(patch_spec.keys()) + additional_layers,
        edit_output=patch_rep,
    ) as td:
        outputs_exp = model(**inp)

    # We report softmax probabilities for the answers_t token predictions of interest.
    probOfGoldToken = torch.softmax(outputs_exp.logits[1:, -1, :], dim=1).mean(dim=0)[answers_t]
    # print('probOfGoldToken: ',probOfGoldToken)

    pred_index = torch.argmax(torch.softmax(outputs_exp.logits[1:, -1, :], dim=1).mean(dim=0)).item()
    # print('pred_index: ',pred_index)

    pred_token = mt.tokenizer.decode([pred_index])
    # print('pred_token: ',pred_token)

    pred_prob = torch.softmax(outputs_exp.logits[1:, -1, :], dim=1).mean(dim=0)[pred_index]
    # print('pred_prob: ',pred_prob)

    result_dict['probOfGoldToken'].append(probOfGoldToken.item())
    result_dict['pred_index'].append(pred_index)
    result_dict['pred_token'].append(pred_token)
    result_dict['pred_prob'].append(pred_prob.item())
    # If tracing all layers, collect all activations together to return.
    if trace_layers is not None:
        all_traced = torch.stack(
            [untuple(td[layer].output).detach().cpu() for layer in trace_layers], dim=2
        )
        return probOfGoldToken, all_traced

    return pred_token

In [ ]:
def calculate_hidden_flow(
    mt, gold_prompt, pred_prompt, subject,result_dict, samples=10, noise=0.1, window=10, kind=None
):
    """
    Runs causal tracing over every token/layer combination in the network
    and returns a dictionary numerically summarizing the results.
    """
    gold_inp = make_inputs(mt.tokenizer, [gold_prompt] * (samples + 1))
    pred_inp = make_inputs(mt.tokenizer, [pred_prompt] * (samples + 1))
    with torch.no_grad():
        answer_t, base_score = [d[0] for d in predict_from_input(mt.model, gold_inp)]
    [answer] = decode_tokens(mt.tokenizer, [answer_t])
    result_dict['gold_index'].append(answer_t)
    result_dict['gold_prob'].append(base_score.item())
    result_dict['gold_token'].append(answer)
    # print('gold_index: ',answer_t)
    # print('gold_prob: ',base_score)
    print('gold_token: ',answer)
    e_range = find_token_range(mt.tokenizer, pred_inp["input_ids"][0], subject)
    pred_token = trace_with_patch(
        mt.model, pred_inp, [], answer_t, e_range,result_dict, noise=noise
    )
    print("pred_token: ", pred_token)
    return pred_token, answer, result_dict

In [ ]:
def plot_hidden_flow(
    mt,
    prompt,
    subject=None,
    samples=10,
    noise=0.1,
    window=10,
    kind=None,
    modelname=None,
    savepdf=None,
    token_to_predict = None
):
    if subject is None:
        subject = guess_subject(prompt)
    result_dict = {'gold_index': [],'gold_prob': [],'gold_token': [], 'pred_index': [],'pred_prob': [],'pred_token': [], 'probOfGoldToken' : []}
    final_text = []
    gold_text = prompt
    pred_text = prompt
    for i in range(token_to_predict):
        pred_token, gold_token, result_dict = calculate_hidden_flow(mt, gold_text, pred_text, subject, result_dict, samples=samples, noise=noise, window=window, kind=kind)
        # print(pred_token)
        pred_text += pred_token
        gold_text += gold_token
        
        # final_text.append(prompt)
    # plot_trace_heatmap(result, savepdf, modelname=modelname)
    return gold_text, text, result_dict




In [ ]:
noise_level = 1
token_to_predict = 60
prompt = """Question: Tom has 10 oranges. Lucy has 5 more. How many oranges do they have? Answer: Let's break it down into multiple steps. Initially, Tom has var1 oranges. Lucy has var2 oranges. So they have (var1+var2) oranges in total. Here, var1 = 10, var2 = 5. So they have (10+5) = 15 oranges in total.
Question: Emma has 15 bananas. Lucy takes away 3 bananas. How many bananas does Emma have now? Answer: Let's think logically. Initially, Emma has var1 bananas. Lucy takes away var2 bananas. So Emma has (var1-var2) bananas in total. Here, var1 = 15, var2 = 3. So Emma has (15-3) = 12 bananas.
Question: Tom has 10 grapes. Lucy adds 6 more. How many grapes do they have now? Answer: Let's break it down into multiple steps. Initially, Tom has var1 grapes. Lucy has var2 grapes. So they have (var1+var2) grapes in total. Here, var1 = 10, var2 = 6. So they have (10+6) = 16 grapes in total.
Question: Emma has 20 apples. Lucy takes away 7 apples. How many apples does Emma have now? Answer: Let's think logically."""
subject = "Emma has 20 apples. Lucy takes away 7 apples."
pred_text, gold_text, result_dict = plot_hidden_flow(mt, prompt, subject = subject, noise=noise_level, token_to_predict=token_to_predict)

In [ ]:
print("Prompt :", prompt)
print("noise_level: ", noise_level)
print("tokens predicted: ", token_to_predict)
print("token noise added: ", subject)

In [ ]:
print("Predicted text after adding noise of ", noise_level, ": ", pred_text)

In [ ]:
print("Predicted text with no noise: ", gold_text)

In [ ]:
gold_prob = result_dict['gold_prob']
probOfGoldToken = result_dict['probOfGoldToken']
gold_token = result_dict['gold_token']

In [ ]:
pred_prob = result_dict['pred_prob']

In [ ]:
pred_token = result_dict['pred_token']

In [ ]:
# importing the required library
import seaborn as sns
import matplotlib.pyplot as plt
 
# read a titanic.csv file
# from seaborn library
# df = sns.load_dataset('titanic')
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x = 'pred_prob', y = 'pred_token', data = result_dict, ci = None, color = 'blue')
ax.set(xlabel='Probability of predicted token', ylabel='Predicted tokens after noise of ' + str(noise_level))
 
# Show the plot
plt.show()

In [ ]:
# importing the required library
import seaborn as sns
import matplotlib.pyplot as plt
 
# read a titanic.csv file
# from seaborn library
# df = sns.load_dataset('titanic')
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x = 'gold_prob', y = 'gold_token', data = result_dict, ci = None, color = 'blue')
ax.set(xlabel='Probability of predicted token', ylabel='Predicted tokens with no noise')
 
# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
   
Women = [115, 215, 250, 200]
Men = [114, 230, 510, 370]
gold_prob = result_dict['gold_prob']
probOfGoldToken = result_dict['probOfGoldToken']
gold_token = result_dict['gold_token']
n=token_to_predict
r = np.arange(n)
width = 0.3
  
plt.figure(figsize=(20, 10))  
plt.bar(r, gold_prob, color = 'r',
        width = width,
        label='Prob of token(no noise model)',align='edge')
plt.bar(r + width, probOfGoldToken, color = 'g',
        width = width,
        label='Prob of token(noisy model)',align='edge')
  
plt.xlabel("Predicted token in no noise model")
plt.ylabel("Probability of token")
# plt.title("Number of people voted in each year")
  
# plt.grid(linestyle='--')
plt.xticks(r + width/2,gold_token, rotation='vertical')
# plt.xticks(rotation=70)
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.autoscale(tight=True)
plt.show()